<a href="https://colab.research.google.com/github/Pranav-2509/NIGHT_VISION_SELF/blob/main/DC_Miniproj_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

In [4]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")
    print (reverse)

    # Read the file and split into lines
    lines = open('%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    print (lines)

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    print (pairs)
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [5]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):

    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
True


FileNotFoundError: [Errno 2] No such file or directory: 'eng-fra.txt'

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [ ]:
import torch
import torch.nn as nn

# Create an embedding layer with vocabulary size 100 and embedding dimension 50
embedding_layer = nn.Embedding(num_embeddings=100, embedding_dim=50)

# Example input indices
input_indices = torch.tensor([[1, 5, 20, 3], [10, 15, 30, 25]])

# Lookup
embedded_vectors = embedding_layer(input_indices)

# Output
print(embedded_vectors)  # Output: torch.Size([2, 4, 50])


In [ ]:
import torch
import torch.nn as nn

# Define input size, hidden size, and sequence length
input_size = 10
hidden_size = 20
sequence_length = 5
batch_size = 3

# Create input tensor
input_data = torch.randn(batch_size, sequence_length, input_size)  # Corrected the input shape

# Define GRU layer
gru_layer = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)

# Initialize hidden state
hidden_state = torch.zeros(1, batch_size, hidden_size)

# Forward pass
output, final_hidden_state = gru_layer(input_data)

# Output shapes
print("Output shape:", output.shape)  # Output shape: torch.Size([3, 5, 20])
print("Final hidden state shape:", final_hidden_state.shape)  # Final hidden state shape: torch.Size([1, 3, 20])


In [ ]:
print (output)
print (final_hidden_state)

In [ ]:
x = torch.tensor([1, 2, 3, 4])

# Unsqueezing along axis 0 (adding a dimension at the beginning)
x_unsqueezed = x.unsqueeze(1)

print("Original tensor:")
print(x)
print("Shape:", x.shape)
print()

print("Tensor after unsqueezing along axis 0:")
print(x_unsqueezed)
print("Shape:", x_unsqueezed.shape)


# *Attention*

I am considering:


1.   batch_size = 3
2.   hidden_size = 4
3.   seq_length = 5
4.   output_vocab = 20
5.   input_vocab = 10
6.   input_size = hidden_size






In [ ]:
batch_size = 3
hidden_size = 4
seq_length = 5
output_vocab = 20
input_size = 4

In [ ]:
gru = nn.GRU (input_size, hidden_size, batch_first=True)
embedding = nn.Embedding (output_vocab, input_size)

In [ ]:
#o/p of the first GRU in Decoder should be of size (1, batch_size, hidden_size)

ip1 = torch.randint (1, 5, (batch_size, 1))
print (ip1)

output = embedding (ip1)
print (output.shape)

output1, hidden = gru(output)  #output1 = (batch_size, input seq_length, hidden_size)
print (output1.shape, hidden.shape)        #hidden =  (1, batch_size, hidden_size)

In [ ]:
#Passing the hidden state through a linear layer
linear_layer = nn.Linear (hidden_size, hidden_size)

hidden2 = linear_layer(hidden)
print (hidden2, hidden, linear_layer.weight, linear_layer.bias)

In [ ]:
#Declaring a Tensor to represent the hidden states of the encoder
hidden_encoder = torch.randint (1, 5 ,(batch_size, seq_length, hidden_size))

In [ ]:
print (hidden_encoder)
hidden_encoder = hidden_encoder.float()

In [ ]:
#passing this through linear activation
hidden_encoder2 = linear_layer (hidden_encoder)
print (hidden_encoder2, hidden_encoder)

In [ ]:
hidden2 = torch.squeeze(hidden2,0)
print (hidden2.shape)

In [ ]:
#Adding a dimension to hidden2
hidden2 = torch.unsqueeze(hidden2, 1)
print (hidden2.shape)

In [ ]:
print (hidden2, hidden_encoder2)

In [ ]:
prior_tanh = hidden_encoder2 + hidden2
print (prior_tanh)

In [ ]:
#then after tanh
linear_layer2 = nn.Linear(hidden_size, 1)

In [ ]:
post_tanh = linear_layer2 (prior_tanh)
print (post_tanh)

In [ ]:
post_softmax = F.softmax (post_tanh, dim=1)

In [ ]:
post_softmax

In [ ]:
hidden_encoder

In [ ]:
prior_concat = hidden_encoder*post_softmax
print (prior_concat)

In [ ]:
prior_concat = torch.sum (prior_concat, dim=1)

In [ ]:
print (prior_concat)

In [ ]:
input_vec = embedding (torch.tensor([[1], [3], [8]]))
print (input_vec)

In [ ]:
a = torch.randn (3, 1, 4)
b = torch.randn(3, 1, 4)
print (a)
print (b)
print (torch.cat((a, b), dim=2))

In [ ]:
prior_concat = prior_concat.unsqueeze(1)
next_input = torch.cat ((input_vec, prior_concat), dim = 1)
print (prior_concat, next_input)

In [ ]:
next_input2 = torch.reshape(next_input, (3, 1, 8))

In [ ]:
next_input2

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)   #To make dim from (3, 5, 1) to (3, 1, 5)
                                                  #Keys are the encoder hidden states
        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)        #Weighted sum over timestamps

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)           #to make the hidden state dim from (1, batch_size, hidden_size) to (batch_size, 1, hidden_size)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

**Training**

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    input_lang, output_lang, pairs = prepareData('eng', 'fra', True)

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    return input_lang, output_lang, train_dataloader

In [ ]:
a = np.zeros ((4, 4), dtype = np.int32)
b = np.zeros ((4, 2), dtype = np.int32)
for i in range (4):
  a[i, :] = i
  b[i, :] = i
train_data = TensorDataset(torch.LongTensor(a).to(device),
                               torch.LongTensor(b).to(device))
train_sampler = RandomSampler (train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=2)


In [ ]:
for batch in train_dataloader:
  print (batch)

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 128
batch_size = 32

input_lang, output_lang, train_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)

train(train_dataloader, encoder, decoder, 80, print_every=5, plot_every=5)

RANDOM

In [ ]:
god = torch.randint (1, 5, (1, 3, 4))
print (god)
goddess = god.permute (1, 0, 2)
print (goddess)

In [ ]:
gru2 = nn.GRU (4, 4, batch_first=True)
hidden_s = torch.randn(3, 1, 4)
present_s = torch.randn(3, 5, 4)
output23, hidden23 = gru2(present_s, hidden_s)


In [ ]:
class Pranav:
  def __init__(self, height, weight):
    self.height = height
    self.weight = weight

  def makeNoise(self, x):
    return x
class Shreya:
  def __init__(self, h, w):
    self.j = Pranav(h, w)
  def forward (self):
    y = self.j(5)
    print (y)
human = Pranav(5, 34)
z = human(5)
print (z)
human2 = Shreya(3, 4)
human2.forward

In [ ]:
class BaseClass:
    def __init__(self, x, y):
        self.x = x
        self.y = y

class DerivedClass(BaseClass):
    def __init__(self, x, y, z, w):
        super(DerivedClass, self).__init__(x, y)#initializer method of the base class
        self.z = z
        self.w = w


d = DerivedClass(1, 2, 3, 4)
d.x

In [ ]:
pairs = [[1, 2], [3, 2], [4, 3], [5, 6]]
for idx, (x, y) in enumerate (pairs):
  print (idx, x, y)